In [ ]:
import base64
import requests
import os
# OpenAI API Key
from dotenv import load_dotenv  
from openai import OpenAI
load_dotenv()

openai_api_key = os.environ.get("INFINI_API_KEY")
openai_base_url = os.environ.get("INFINI_BASE_URL")
client = OpenAI(api_key=openai_api_key, base_url=openai_base_url)

: 

In [3]:
# Function to encode the image
def encode_image(image_input):
    import io
    from PIL import Image
    
    if isinstance(image_input, str):
        # 处理文件路径
        with open(image_input, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    else:
        # 处理PIL Image对象
        buffer = io.BytesIO()
        image_input.save(buffer, format="PNG")
        return base64.b64encode(buffer.getvalue()).decode('utf-8')


In [4]:
import base64
import requests
from PIL import Image
# Getting the base64 string
example1_path = "male\\391.png"
image_1 = Image.open(example1_path)
flipped_image_exp1 = image_1.transpose(Image.FLIP_LEFT_RIGHT)
base64_example1 = encode_image(flipped_image_exp1)

example2_path = "male\\111.png"
image_2 = Image.open(example2_path)
flipped_image_exp2 = image_2.transpose(Image.FLIP_LEFT_RIGHT)
base64_example2 = encode_image(flipped_image_exp2)

example3_path = "test\\female\\385.png"
image_3 = Image.open(example3_path)
flipped_image_exp3 = image_3.transpose(Image.FLIP_LEFT_RIGHT)
base64_example3 = encode_image(flipped_image_exp3)

In [ ]:
def get_prompt(image_path):
  image = Image.open(image_path)
  flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)
  
  base64_image = encode_image(flipped_image)
  
  response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
      {
        "role": "system",
        "content": """You are a helpful prompt generator. I have a pair of couple avatar, you need to observe one of the images and predict the feature of the other couple avatar to train a diffusion model. You should prompt it out with booru tags. One pair of couple avatars consists of a male and female avatar with same oritation. Do not directly describe the provided figure. Your description should be consistent with the gender of your predicted avatar.
                      Try your best to predict the features of the other corresponding couple avatar, you should at least point out the gender, species, body orientation, position. You can also add a few guess features such as clothing, expression."""
      },
      {"role":"user", "content":"Only write a prompt to predict the features of the other couple avatar in booru tags. Here are two examples. "}, 
      {
        "role": "user",
        "content": [
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_example1}",
              "detail": "high"
            },
          },
          {"type": "text", "text": "The response should be like: female, human, facing right, center of the image, white background, pick hair, pink eyes."},
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_example2}",
              "detail": "high"
            },
          },
          {"type": "text", "text": "The response should be like: female, human, face left, center of the image, fleshcolor background, black hair."},
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_example3}",
              "detail": "high"
            },
          },
          {"type": "text", "text": "The response should be like: male, human, face forward, center of the image, smiling, black robe."},
          {"type": "text", "text": "Instruction:Now Read this image carefully and write a prompt for it like examples."},
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}",
              "detail": "high"
            },
          },
        ],
      }
    ],
    max_tokens=5000,
  )
  return response.choices[0]
  #Merge duplicate tags into one, for example, give \"black dress\" instead of \"dress\". Give exactly 1 tag for each feature, for instance, give \"white and blue dress\" instead of \"white dress\" and \"blue dress\"separately. Give your prior first to on the figure then the environment. 


In [6]:
import os
import re

def list_files(directory):
    file_paths = []  # 用于存储所有文件的路径
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            file_paths.append(file_path)
    
    # 使用自定义排序函数，提取文件名中的数字并按数字大小排序
    def numerical_sort(file_path):
        # 提取文件名（不包括路径）
        filename = os.path.basename(file_path)
        # 提取文件名中的数字部分
        numbers = re.findall(r'\d+', filename)
        # 如果找到数字，则返回第一个数字的整数值；否则返回0
        return int(numbers[0]) if numbers else 0
    
    # 对文件路径列表进行排序
    return sorted(file_paths, key=numerical_sort)

In [12]:
directory = 'images\\male'
output_dic = 'test\\male_remaining'
sorted_files = list_files(directory)
for file_path in sorted_files:
    print(file_path)
    


images\male\0.jpeg
images\male\1.jpeg
images\male\2.jpeg
images\male\3.jpeg
images\male\4.jpeg
images\male\5.jpeg
images\male\6.jpeg
images\male\7.jpeg
images\male\8.jpeg
images\male\9.jpeg
images\male\10.jpeg
images\male\11.jpeg
images\male\12.jpeg
images\male\13.jpeg
images\male\14.jpeg
images\male\15.jpeg
images\male\16.jpeg
images\male\17.jpeg
images\male\18.jpeg
images\male\19.jpeg
images\male\20.jpeg
images\male\21.jpeg
images\male\22.jpeg
images\male\23.jpeg
images\male\24.jpeg
images\male\25.jpeg
images\male\26.jpeg
images\male\27.jpeg
images\male\28.jpeg
images\male\29.jpeg
images\male\30.jpeg
images\male\31.jpeg
images\male\32.jpeg
images\male\33.jpeg
images\male\34.jpeg
images\male\35.jpeg
images\male\36.jpeg
images\male\37.jpeg
images\male\38.jpeg
images\male\39.jpeg
images\male\40.jpeg
images\male\41.jpeg
images\male\42.jpeg
images\male\43.jpeg
images\male\44.jpeg
images\male\45.jpeg
images\male\46.jpeg
images\male\47.jpeg
images\male\48.jpeg
images\male\49.jpeg
images\mal

In [10]:

def convert_rgba_to_rgb(image_path, output_path, background_color=(255, 255, 255)):
    image = Image.open(image_path).convert("RGBA")
    background = Image.new("RGB", image.size, background_color)
    combined = Image.alpha_composite(background.convert("RGBA"), image)
    
    combined.convert("RGB").save(output_path, "PNG")
def is_image_file(filename):
    image_extensions = ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.tiff', '.webp']
    ext = os.path.splitext(filename)[1].lower()
    return ext in image_extensions


In [20]:

from PIL import Image
i=0
for file in sorted_files:
    if(is_image_file(file)):
        print(f"Processing file: {file}")
        convert_rgba_to_rgb(file, output_dic+f"/{i}.png")
        i=i+1

Processing file: images\male\0.jpeg
Processing file: images\male\1.jpeg
Processing file: images\male\2.jpeg
Processing file: images\male\3.jpeg
Processing file: images\male\4.jpeg
Processing file: images\male\5.jpeg
Processing file: images\male\6.jpeg
Processing file: images\male\7.jpeg
Processing file: images\male\8.jpeg
Processing file: images\male\9.jpeg
Processing file: images\male\10.jpeg
Processing file: images\male\11.jpeg
Processing file: images\male\12.jpeg
Processing file: images\male\13.jpeg
Processing file: images\male\14.jpeg
Processing file: images\male\15.jpeg
Processing file: images\male\16.jpeg
Processing file: images\male\17.jpeg
Processing file: images\male\18.jpeg
Processing file: images\male\19.jpeg
Processing file: images\male\20.jpeg
Processing file: images\male\21.jpeg
Processing file: images\male\22.jpeg
Processing file: images\male\23.jpeg
Processing file: images\male\24.jpeg
Processing file: images\male\25.jpeg
Processing file: images\male\26.jpeg
Processing 

In [13]:
all_files=list_files(output_dic)
prompt = get_prompt(all_files[0])
print(prompt.message.content)
print(all_files[0])

female, human, facing towards the center, positioned lower in the image, monochrome background, long hair.
test\male_remaining\567.png


In [14]:
directory="new_samples1"
for file in all_files:
    if is_image_file(file):
        name = file[:-4]
        file_path = f'{name}.txt'
        prompt = get_prompt(file)
        print(prompt.message.content)
        #final_prompt = get_final_prompt(prompt.message.content)
        #print(final_prompt.message.content + "\n")
        with open(file_path, 'w') as file:
            file.write(prompt.message.content)
        print(file_path)

female, human, facing left, right side of the image, black and white, long hair, relaxed expression.
test\male_remaining\567.txt
female, human, facing forward, black hair, closed eyes, white background, slight blush, short hair.
test\male_remaining\568.txt
female, human, facing right, upper body, black hair, smiling expression, dark clothing, black and white background
test\male_remaining\569.txt
female, human, facing right, lying down, black hair, closed eyes, dark blue background.
test\male_remaining\570.txt
female, human, facing left, center of the image, brown hoodie, bear ears, long brown hair.
test\male_remaining\571.txt
male, human, facing left, center of the image, monochrome background, dark hair, wearing a striped outfit, serious expression.
test\male_remaining\572.txt
female, human, facing forward, right of the image, black and white, shoulder-length hair, expressing cheerfulness, hand on shoulder.
test\male_remaining\573.txt
female, human, facing right, center of the image,